In [30]:
import os
import mlflow
import numpy as np
import pandas as pd

from math import sqrt
from joblib import delayed
from joblib import Parallel

from warnings import catch_warnings
from warnings import filterwarnings
from multiprocessing import cpu_count
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from matplotlib import pyplot
import matplotlib.dates as mdates

from pathlib import Path
from functools import reduce
from datetime import datetime

from prophet import Prophet

import random
from scipy.stats import ttest_ind
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression, RidgeCV, Ridge
from regressors import stats
from mlxtend.feature_selection import SequentialFeatureSelector as sfs


In [31]:
df = pd.read_csv('./kr_massnahmen_oberkategorien.csv', converters={'ags2': str, 'ags5': str} )
df = df.drop(['_id'], axis=1)

In [32]:
di = {'M01a': 'meeting_restrictions_private', 
      'M01b': 'meeting_restrictions_public',
      'M02a': 'secondary_schools',
     'M02b': 'primary_schools',
     'M03': 'daycare_centers',
     'M04': 'public_and_indoor_events',
     'M05': 'public_and_outdoor_events',
     'M06': 'culture_and_educational_institution',
     'M07': 'wholesale_retail',
     'M08': 'gastronomy',
     'M09': 'service_and_crafts',
     'M10': 'nightlife',
     'M11': 'accommodation',
     'M12': 'sport_indoor',
     'M13': 'sport_outdoor',
     'M14': 'travel_domestic',
     'M15': 'travel_abroad',
     'M16': 'mask_requirement',
     'M17': 'job_restriction',
     'M18': 'curfew',
     'M19': 'capacity_limitation_in_public_traffic',
     'M20': 'distance_regulation ',
     'M21': 'test_measures'}
df = df.replace({"m_code": di})

In [33]:
df = df.melt(id_vars=['m_code','ags5','ags2','bundesland','kreis'], var_name='date', value_name='vals')

In [34]:
df = df.pivot(index=['ags5','ags2','bundesland','kreis','date'], columns='m_code', values='vals')
df = df.reset_index(drop=False)

In [35]:
df['date'] = df['date'].str[1:]
df['date'] = pd.to_datetime(df['date'], format = '%Y%m%d')

In [36]:
df

m_code,ags5,ags2,bundesland,kreis,date,accommodation,capacity_limitation_in_public_traffic,culture_and_educational_institution,curfew,daycare_centers,...,public_and_indoor_events,public_and_outdoor_events,secondary_schools,service_and_crafts,sport_indoor,sport_outdoor,test_measures,travel_abroad,travel_domestic,wholesale_retail
0,01001,01,Schleswig-Holstein,"Flensburg, Stadt",2020-03-01,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,01001,01,Schleswig-Holstein,"Flensburg, Stadt",2020-03-02,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,01001,01,Schleswig-Holstein,"Flensburg, Stadt",2020-03-03,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,01001,01,Schleswig-Holstein,"Flensburg, Stadt",2020-03-04,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,01001,01,Schleswig-Holstein,"Flensburg, Stadt",2020-03-05,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176836,16077,16,Thüringen,Altenburger Land,2021-05-11,1,0,1,1,1,...,1,1,0,1,1,1,1,0,0,1
176837,16077,16,Thüringen,Altenburger Land,2021-05-12,1,0,1,1,1,...,1,1,0,1,1,1,1,0,0,1
176838,16077,16,Thüringen,Altenburger Land,2021-05-13,1,0,1,1,1,...,1,1,0,1,1,1,1,0,0,1
176839,16077,16,Thüringen,Altenburger Land,2021-05-14,1,0,1,1,1,...,1,1,0,1,1,1,1,0,0,1


In [38]:
df.to_csv('./covid_measurements_main.csv', index = False)